### === **SelectKBest for Feature Selection to optimize feature subsets for various ML classifiers** ===

-----------------------------------------------------------------------------------------------

**1. Stable Feature Pre-selection:**

* Performed initial feature selection using **bootstrapped LASSO logistic regression** (`penalty='l1'`, `solver='liblinear'`, `cv=5`, `max_iter=5000`), repeated for **100 bootstrap samples** (`n_bootstrap=100`). Features were retained if selected in **≥70%** of bootstraps (`freq_threshold=0.7`), ensuring stability against sampling variance.


**2. Metaheuristic Feature Selection:**

* Applies **Particle Swarm Optimization (PSO)** for further feature selection from the stable set, using a population of **20 particles** (`n_particles=20`) and **50 iterations** (`max_iter=50`). Each particle represented a binary feature mask, and the swarm was optimized to maximize mean ROC AUC over 5-fold stratified cross-validation (`StratifiedKFold(n_splits=5, shuffle=True, random_state=42)`).


**3. Comprehensive Classifier Comparison:**

* Evaluated a broad suite of classifiers:

  * **Logistic Regression** (`max_iter=1000`), with grid search over `C=[0.001, 0.01, 0.1, 1, 10]` and `penalty=['l1', 'l2']`

  * **Gaussian Naive Bayes** (default parameters)

  * **Support Vector Machines** (linear and RBF kernels, `C=[0.01, 0.1, 1, 10]`, `gamma=['scale', 'auto']`)

  * **Decision Tree** (`max_depth=[3, 5, 7]`, `min_samples_split=[5, 10]`, `min_samples_leaf=[2, 4]`)

  * **Random Forest** (`n_estimators=[100, 200]`, `max_depth=[5, 10]`, `min_samples_split=[5, 10]`, `min_samples_leaf=[2, 4]`)

  * **VotingClassifier** ensemble combining top-tuned base models with soft voting.

* Hyperparameters were tuned for each classifier using **GridSearchCV** and 5-fold stratified cross-validation, optimizing for ROC AUC.


**4. Class Imbalance and Data Integrity Handling:**

* Maintained **class distribution balance** in all data splits using stratified sampling, both in train/test partitioning (`test_size=0.2`) and during cross-validation.

* For bootstrapping and CI estimation, ensured that each resample included both classes—skipping samples otherwise to avoid invalid AUC calculations.


**5. Feature Scaling and Pipeline Safety:**

* Applied **feature standardization** (`StandardScaler()`) within all pipelines, fitting scalers only on training data to prevent information leakage.

* Model pipelines (`make_pipeline(StandardScaler(), classifier)`) ensured consistent preprocessing during evaluation and prediction.


**6. Model Evaluation & Uncertainty Quantification:**

* Reported **ROC AUC and accuracy** for both training and testing sets at each PSO iteration.

* Computed **bootstrapped confidence intervals** for test AUC (`n_bootstrap=1000`, `ci=0.95`), resampling test predictions to quantify model uncertainty and generalization performance.


**7. Visualization and Monitoring:**

* Plotted **train/test AUC trends** across PSO iterations for convergence analysis.

* Displayed final **ROC curves** for both training and testing sets, including mean AUC and 95% confidence intervals.


---

##### === IMPORTS ===

In [7]:
import numpy as np                                                                            # Core numerical computation library for arrays and matrix operations
import pandas as pd                                                                           # Library for data manipulation and analysis (tabular data, DataFrames)
from sklearn.base import clone                                                                # For cloning estimators (useful when building pipelines or doing cross-validation)
import matplotlib.pyplot as plt                                                               # For plotting graphs (e.g., ROC curves, feature importances, etc.)
from scipy.stats import bootstrap                                                             # For statistical bootstrap resampling (scipy's bootstrap is for CI, sklearn's resample for random sampling)
from sklearn.utils import resample                                                            # Utility to randomly resample datasets (e.g., for bootstrapping in ML)
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold        # For splitting data, cross-validation, and stratified folds (for imbalanced classes)
from sklearn.preprocessing import StandardScaler                                              # For scaling features to zero mean/unit variance (important for many ML algorithms)
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score                          # For evaluating model performance: ROC AUC, ROC curve points, accuracy

# === CLASSIFIERS ===

from sklearn.naive_bayes import GaussianNB                                                    # Naive Bayes classifier for classification tasks
from sklearn.svm import SVC                                                                   # Support Vector Classifier (linear & nonlinear SVMs)
from sklearn.tree import DecisionTreeClassifier                                               # Decision Tree classifier (nonlinear, interpretable ML model)
from sklearn.ensemble import RandomForestClassifier, VotingClassifier                         # Random Forest classifier (ensemble of Decision Trees)
from sklearn.pipeline import make_pipeline, Pipeline                                          # For creating machine learning pipelines (combining preprocessing + models)
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV                     # Logistic Regression (standard and cross-validated, for classification)

# === GRID SEARCH HYPERPARAMETER TUNING ===

from sklearn.model_selection import GridSearchCV                                              # For exhaustive grid search over hyperparameters with cross-validation
from sklearn.feature_selection import SelectKBest, f_classif                                   # For univariate feature selection (SelectKBest with ANOVA F-value)

---

##### === LOAD AND PREPROCESS DATA ===

In [8]:
def load_data():
    # Load radiomics and clinical CSV files into DataFrames
    radiomics = pd.read_csv("./HNC-Prospective-Radiomics-305.csv")
    clinical = pd.read_csv("./proceed_radiomics_128.csv")

    print(f"Initial clinical data: {len(clinical)} patients")
    # print(f"Unique locations in clinical data: {clinical['Location'].value_counts()}")

    # Filter clinical data to only include specific tumor locations
    # clinical = clinical[clinical["Location"].isin(['Larynx', 'Tonsil', 'Hypopharynx', 'Oropharynx', 'BOT', 'Other'])]
    # print(f"After location filtering: {len(clinical)} patients")

    # Standardize 'research_subject_uid' in radiomics by keeping only the part before "_"
    radiomics["research_subject_uid"] = radiomics["research_subject_uid"].apply(lambda x: x.split("_")[0])

    # Remove any leading/trailing spaces from 'Project ID' in clinical data
    clinical["Project ID"] = clinical["Project ID"].str.strip()

    # Filter radiomics to only keep rows with research_subject_uid present in clinical Project IDs
    radiomics_filtered = radiomics[radiomics["research_subject_uid"].isin(clinical["Project ID"])]
    
    # Filter clinical to only keep rows with Project ID present in radiomics research_subject_uid
    clinical_filtered = clinical[clinical["Project ID"].isin(radiomics["research_subject_uid"])]

    print(f"Final matched data: {len(clinical_filtered)} patients")

    # Sort both DataFrames by their ID columns and reset their indices
    radiomics_filtered = radiomics_filtered.sort_values(by="research_subject_uid").reset_index(drop=True)
    clinical_filtered = clinical_filtered.sort_values(by="Project ID").reset_index(drop=True)

    # Return the filtered and aligned DataFrames for further processing
    return radiomics_filtered, clinical_filtered


def get_radiomics_columns(data):
    """
    Returns the columns from 'original_shape_Elongation' to 'original_ngtdm_Strength'.
    These typically represent the set of radiomics features you want to extract.
    """
    start_column = "original_shape_Elongation"
    end_column = "original_ngtdm_Strength"
    start_idx = data.columns.get_loc(start_column)
    end_idx = data.columns.get_loc(end_column) + 1  # +1 to include the end column itself
    return data.columns[start_idx:end_idx]

---

##### === Define hyperparameter grids for each classifier ===

In [9]:
# These grids help tune the model to avoid overfitting by optimizing regularization and other key parameters.
all_grid_params = {
    'LogisticRegression': {
        'C': [1e-4, 3e-4, 1e-3, 3e-3, 1e-2, 3e-2, 0.1, 0.3, 1, 3, 10],          # Regularization strength (lower = stronger regularization)
        'penalty': ['l1', 'l2'],                                                # Type of regularization: L1 (Lasso), L2 (Ridge)
        'solver': ['liblinear'],                                                # Solver that supports both l1 and l2
        'class_weight': [None, "balanced"],
        'max_iter': [1000],
    },
    'GaussianNB': {},                                                           # No tunable hyperparameters for basic Naive Bayes
    'SVC': [
        # Linear SVC
        {
            'C': [1e-3, 1e-2, 0.1, 1, 10],                                      # Regularization strength
            'kernel': ['linear'],                                               # Linear kernel
            'probability': [True],                                              # Needed for probability predictions (e.g., ROC AUC)
            'class_weight': [None, "balanced"],
        },
        # RBF SVC
        {
            'C': [1e-3, 1e-2, 0.1, 1, 10],                                      # Regularization strength
            'gamma': ['scale', 'auto', 1e-3, 1e-2, 1e-1],                       # Kernel coefficient for RBF
            'kernel': ['rbf'],                                                  # RBF (nonlinear) kernel
            'probability': [True],                                              # Probability estimates
            'class_weight': [None, "balanced"],
        }
    ],
    'DecisionTreeClassifier': {
        'criterion': ['gini', 'entropy'],                                       # Split quality metric
        'max_depth': [2, 3, 4, 5, 7],                                           # Controls tree depth (regularization)
        'min_samples_split': [5, 10, 15],                                       # Minimum samples required to split a node
        'min_samples_leaf': [2, 4, 6],                                          # Minimum samples per leaf node
        'max_features': ['sqrt', 'log2', None],                                 # Number of features considered at each split
        'class_weight': [None, "balanced"],
        'random_state': [42],
    },
    'RandomForestClassifier': {
        'n_estimators': [100, 200, 400],                                        # Number of trees
        'max_depth': [3, 5, 7, 10],                                             # Maximum depth of trees
        'min_samples_split': [5, 10],                                           # Minimum samples to split an internal node
        'min_samples_leaf': [2, 4],                                             # Minimum samples at a leaf node
        'max_features': ['sqrt', 'log2'],                                       # Number of features considered at each split
        'bootstrap': [True],                                                    # Use bootstrap samples
        'n_jobs': [-1],                                                         # Use all available CPU cores for parallel processing
        'class_weight': [None, "balanced", "balanced_subsample"],
        'random_state': [42],
    },
}



# === Utility function to tune a classifier's hyperparameters using grid search and cross-validation ===

def get_tuned_model(classifier, X_train, y_train):
    name = classifier.__class__.__name__                                 # Get class name as a string (e.g., 'LogisticRegression')
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)      # 5-fold stratified cross-validation
    
    # Special handling for SVC as its grid is a list (to support both linear & RBF kernels)
    if name == 'SVC':
        grid = GridSearchCV(
            clone(classifier),            # Clone base estimator to avoid data leakage between folds
            all_grid_params['SVC'],
            scoring='roc_auc',            # Use ROC AUC for model selection (works for imbalanced data)
            cv=cv,                        # Use stratified k-fold
            n_jobs=-1,                    # Use all CPU cores
            refit=True,
        )
        grid.fit(X_train, y_train)       # Fit grid search
        return grid.best_estimator_      # Return the model with best hyperparameters

    # For all other classifiers with defined grid parameters
    elif name in all_grid_params and all_grid_params[name]:
        grid = GridSearchCV(
            clone(classifier),
            all_grid_params[name],
            scoring='roc_auc',
            cv=cv,
            n_jobs=-1,
            refit=True,
        )
        grid.fit(X_train, y_train)
        return grid.best_estimator_

    # If no hyperparameters to tune (e.g., GaussianNB), return the original classifier
    return classifier

---

##### === BOOTSTRAP SelectKBest FEATURE SELECTION ===

In [10]:
def selectkbest_feature_selection(X, y, k_values=None, cv=5, scoring='roc_auc', random_state=42):
    """
    Selects optimal k features using SelectKBest with f_classif scoring and cross-validation.
    
    Parameters:
    - X: feature matrix (numpy array)
    - y: target labels
    - k_values: list of k values to test (if None, uses default range)
    - cv: number of cross-validation folds
    - scoring: scoring metric for model selection
    - random_state: for reproducibility

    Returns:
    - best_k: optimal number of features
    - selected_features_idx: indices of selected features
    - cv_scores: cross-validation scores for each k
    """
    
    if k_values is None:
        # Default k values to test (adjust based on your feature count)
        max_features = min(X.shape[1], 50)  # Don't test more than 50 or total features
        k_values = [5, 10, 15, 20, 25, 30, 40, 50]
        k_values = [k for k in k_values if k <= max_features]
    
    print(f"Testing k values: {k_values}")
    
    best_score = 0
    best_k = k_values[0]
    cv_scores = {}
    
    # Test each k value using cross-validation
    for k in k_values:
        print(f"\nTesting k={k}...")
        
        # SelectKBest with f_classif (ANOVA F-test)
        selector = SelectKBest(score_func=f_classif, k=k)
        X_selected = selector.fit_transform(X, y)
        
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', LogisticRegression(max_iter=1000, random_state=random_state))
        ])
        
        # Cross-validation scores
        scores = cross_val_score(pipeline, X_selected, y, cv=cv, scoring=scoring, n_jobs=-1)
        mean_score = scores.mean()
        std_score = scores.std()
        
        cv_scores[k] = {'mean': mean_score, 'std': std_score, 'scores': scores}
        
        print(f"  CV Score: {mean_score:.4f} (±{std_score:.4f})")
        
        # Update best k if this is better
        if mean_score > best_score:
            best_score = mean_score
            best_k = k
    
    print(f"\nBest k: {best_k} with CV score: {best_score:.4f}")
    
    # Fit final selector with best k on full training data
    final_selector = SelectKBest(score_func=f_classif, k=best_k)
    final_selector.fit(X, y)
    selected_features_idx = final_selector.get_support(indices=True)
    
    print(f"Selected {len(selected_features_idx)} features: {selected_features_idx}")
    
    return best_k, selected_features_idx, cv_scores

---

##### === Evaluate model with classifier function ===

In [11]:
def evaluate_model_with_classifier(X_train, X_test, y_train, y_test, selected_features, feature_names, classifier):
    """
    Trains and evaluates a given classifier using only the features selected by a feature selection algorithm.
    Prints Train/Test AUC and Accuracy.
    
    Parameters:
        X_train, X_test: Feature matrices (numpy arrays)
        y_train, y_test: Labels
        selected_features: Binary vector indicating which features to use
        feature_names: List of feature names (not used here, but useful for further reporting)
        classifier: Classifier instance (e.g., LogisticRegression)
    """
    
    # Select only the features chosen by the feature selection mask
    X_train_sel = X_train[:, selected_features == 1]
    X_test_sel = X_test[:, selected_features == 1]

    # If no features were selected, print a warning and stop
    if X_train_sel.shape[1] == 0:
        print("\n⚠️ No features selected. Cannot train classifier.")
        return

    # Build pipeline: scale features then fit classifier
    model = make_pipeline(StandardScaler(), classifier)
    model.fit(X_train_sel, y_train)  # Train the model on selected features

    # Get predicted probabilities or decision function for AUC
    y_train_proba = model.predict_proba(X_train_sel)[:, 1] if hasattr(model, 'predict_proba') else model.decision_function(X_train_sel)
    y_test_proba = model.predict_proba(X_test_sel)[:, 1] if hasattr(model, 'predict_proba') else model.decision_function(X_test_sel)

    # Get predicted labels for accuracy
    y_train_pred = model.predict(X_train_sel)
    y_test_pred = model.predict(X_test_sel)

    # Print evaluation metrics for train and test sets
    print(f"\n✅ Results for {classifier.__class__.__name__}:")
    print(f"Train AUC: {roc_auc_score(y_train, y_train_proba):.4f}")
    print(f"Train Accuracy: {accuracy_score(y_train, y_train_pred):.4f}")
    print(f"Test AUC: {roc_auc_score(y_test, y_test_proba):.4f}")
    print(f"Test Accuracy: {accuracy_score(y_test, y_test_pred):.4f}")


---

##### === Computes a bootstrapped confidence interval for ROC AUC of the final model ===

In [12]:
def bootstrap_final_model_auc_ci(
    y_true,
    y_pred_proba,
    n_bootstrap: int = 1000,
    ci: float = 0.95,
    random_state: int = 42,
    min_valid: int = 200,
):
    """
    Stratified bootstrap CI for ROC AUC on small/imbalanced test sets.
    - Preserves class counts in each bootstrap sample (positives/negatives drawn separately).
    - Avoids invalid replicates with a single class.
    - Returns mean AUC and (lower, upper) percentile CI.
    
    Parameters
    ----------
    y_true : array-like
        True binary labels. If not exactly {0,1}, they will be remapped to {0,1} by ordering.
    y_pred_proba : array-like
        Predicted probabilities or decision scores (higher = more likely positive).
    n_bootstrap : int, default=2000
        Number of bootstrap replicates.
    ci : float, default=0.95
        Confidence level (e.g., 0.95 for 95% CI).
    random_state : int, default=42
        Seed for reproducibility.
    min_valid : int, default=200
        Minimum recommended number of valid bootstrap replicates for a stable CI.
    
    Returns
    -------
    mean_auc : float
    lower : float
    upper : float
    valid : int
        Number of valid bootstrap replicates used.
    """

    rng = np.random.default_rng(random_state)

    # Convert to numpy arrays
    y_true = y_true.values if hasattr(y_true, "values") else np.asarray(y_true)
    y_pred_proba = y_pred_proba.values if hasattr(y_pred_proba, "values") else np.asarray(y_pred_proba)

    # Basic checks
    if y_true.shape[0] != y_pred_proba.shape[0]:
        raise ValueError("y_true and y_pred_proba must have the same number of samples.")

    uniq = np.unique(y_true)
    if uniq.size < 2:
        raise ValueError("AUC undefined: test set has a single class.")
    if uniq.size > 2:
        # Remap to binary by ordering (largest label -> positive class)
        # This allows labels like {-1, +1} or {0, 2}
        pos_label = uniq.max()
        y_true = (y_true == pos_label).astype(int)
        uniq = np.array([0, 1])

    # Class counts
    n_pos = int((y_true == 1).sum())
    n_neg = int((y_true == 0).sum())
    if min(n_pos, n_neg) < 3:
        print(f"Warning: very few positives/negatives (pos={n_pos}, neg={n_neg}); CI will be unstable.")

    # Index pools by class
    pos_idx = np.where(y_true == 1)[0]
    neg_idx = np.where(y_true == 0)[0]

    aucs = []
    for _ in range(n_bootstrap):
        # Stratified resample: draw with replacement within each class
        b_pos = rng.choice(pos_idx, size=n_pos, replace=True)
        b_neg = rng.choice(neg_idx, size=n_neg, replace=True)
        b_idx = np.concatenate([b_pos, b_neg])
        rng.shuffle(b_idx)  # order doesn't matter for AUC, but good practice

        try:
            aucs.append(roc_auc_score(y_true[b_idx], y_pred_proba[b_idx]))
        except ValueError:
            # Extremely unlikely with stratified sampling; keep guard anyway
            continue

    valid = len(aucs)
    if valid == 0:
        print("Error: No valid bootstrap samples generated.")
        return None, None, None, 0
    if valid < min_valid:
        print(f"Warning: Only {valid} valid bootstrap samples (min recommended {min_valid}). CI may be unreliable.")

    # Percentile CI
    lower = float(np.percentile(aucs, (1 - ci) / 2 * 100))
    upper = float(np.percentile(aucs, (1 + ci) / 2 * 100))
    mean_auc = float(np.mean(aucs))

    print(f"Bootstrapped {int(ci*100)}% CI for Final Model Test AUC: {mean_auc:.4f} [{lower:.4f}, {upper:.4f}] "
          f"(valid reps = {valid})")
    return mean_auc, lower, upper, valid


---

##### === `__main__` integration block with Bootstrap SelectKBest ===

In [14]:
if __name__ == "__main__":
    # ---- Load data and extract radiomics features/labels ----
    radiomics_data, clinical_data = load_data()
    radiomics_cols = get_radiomics_columns(radiomics_data)
    X_raw = radiomics_data[radiomics_cols].values
    y = clinical_data["LocoRegeonalRecurrence"].values

    # Print dataset information
    print("DATASET INFORMATION:")
    print(f"Total samples: {len(X_raw)}")
    print(f"Total features: {X_raw.shape[1]}")
    print(f"Class distribution: LRR={np.sum(y)}, Non-LRR={len(y)-np.sum(y)}")
    print(f"LRR percentage: {100*np.sum(y)/len(y):.1f}%")

    # ===============================================================
    # 1) Hold-out TEST set (20%), keep untouched till the very end
    # ===============================================================
    X_temp, X_test_raw, y_temp, y_test = train_test_split(
        X_raw, y, test_size=0.2, stratify=y, random_state=42
    )

    print(f"\nDATA SPLIT:")
    print(f"Train/val samples: {len(X_temp)}")
    print(f"Test samples: {len(X_test_raw)}")
    print(f"Train/val class dist: LRR={np.sum(y_temp)}, Non-LRR={len(y_temp)-np.sum(y_temp)}")
    print(f"Test class dist: LRR={np.sum(y_test)}, Non-LRR={len(y_test)-np.sum(y_test)}")

    # ===============================================================
    # 2) SELECTKBEST Feature Selection with Cross-Validation
    # ===============================================================
    print(f"\n{'='*60}")
    print("SELECTKBEST FEATURE SELECTION WITH 5-FOLD CROSS-VALIDATION")
    print(f"{'='*60}")

    # Define classifiers for evaluation
    classifiers = [
        LogisticRegression(max_iter=1000),
        GaussianNB(),
        SVC(kernel='linear', probability=True),
        SVC(kernel='rbf', probability=True),
        DecisionTreeClassifier(random_state=42),
        RandomForestClassifier(random_state=42, n_jobs=-1)
    ]

    # 5-fold stratified cross-validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    # Storage for cross-validation results
    cv_results = {}
    for clf in classifiers:
        cv_results[clf.__class__.__name__] = {
            'train_aucs': [], 'val_aucs': [], 'train_accs': [], 'val_accs': [], 'n_features': []
        }

    fold_selected_features = []  # Store selected features for each fold
    fold_best_k_values = []      # Store best k for each fold
    fold_k_cv_scores = []        # Store k optimization scores for each fold

    # Cross-validation loop
    for fold_id, (train_idx, val_idx) in enumerate(skf.split(X_temp, y_temp), 1):
        print(f"\nFOLD {fold_id}/5")
        print("-" * 40)
        
        X_train_fold, X_val_fold = X_temp[train_idx], X_temp[val_idx]
        y_train_fold, y_val_fold = y_temp[train_idx], y_temp[val_idx]

        # Standardize features (fit on train, transform both train and val)
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_fold)
        X_val_scaled = scaler.transform(X_val_fold)

        # SelectKBest feature selection on training fold
        print(f"Running SelectKBest feature selection...")
        best_k, selected_features_idx, k_cv_scores = selectkbest_feature_selection(
            X_train_scaled, y_train_fold, 
            k_values=None,  # Use default k values
            cv=3,           # Use 3-fold CV within each training fold
            scoring='roc_auc'
        )
        
        fold_selected_features.append(selected_features_idx)
        fold_best_k_values.append(best_k)
        fold_k_cv_scores.append(k_cv_scores)
        n_selected = len(selected_features_idx)
        print(f"Selected {n_selected} features out of {X_train_scaled.shape[1]} (k={best_k})")

        if n_selected == 0:
            print("⚠️ No features selected in this fold. Skipping evaluation.")
            continue

        # Apply feature selection
        X_train_selected = X_train_scaled[:, selected_features_idx]
        X_val_selected = X_val_scaled[:, selected_features_idx]

        # Evaluate each classifier with selected features
        for clf in classifiers:
            print(f"\nEvaluating {clf.__class__.__name__}...")
            
            # Tune hyperparameters on selected features
            tuned_clf = get_tuned_model(clf, X_train_selected, y_train_fold)
            tuned_clf.fit(X_train_selected, y_train_fold)

            # Predictions
            if hasattr(tuned_clf, 'predict_proba'):
                y_train_proba = tuned_clf.predict_proba(X_train_selected)[:, 1]
                y_val_proba = tuned_clf.predict_proba(X_val_selected)[:, 1]
            else:
                y_train_proba = tuned_clf.decision_function(X_train_selected)
                y_val_proba = tuned_clf.decision_function(X_val_selected)

            y_train_pred = tuned_clf.predict(X_train_selected)
            y_val_pred = tuned_clf.predict(X_val_selected)

            # Calculate metrics
            train_auc = roc_auc_score(y_train_fold, y_train_proba)
            val_auc = roc_auc_score(y_val_fold, y_val_proba)
            train_acc = accuracy_score(y_train_fold, y_train_pred)
            val_acc = accuracy_score(y_val_fold, y_val_pred)

            # Store results
            cv_results[clf.__class__.__name__]['train_aucs'].append(train_auc)
            cv_results[clf.__class__.__name__]['val_aucs'].append(val_auc)
            cv_results[clf.__class__.__name__]['train_accs'].append(train_acc)
            cv_results[clf.__class__.__name__]['val_accs'].append(val_acc)
            cv_results[clf.__class__.__name__]['n_features'].append(n_selected)

            print(f"  Train AUC: {train_auc:.4f}, Val AUC: {val_auc:.4f}")
            print(f"  Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

    # ===============================================================
    # 3) Build consensus features from cross-validation
    # ===============================================================
    print(f"\n{'='*60}")
    print("BUILDING CONSENSUS FEATURES FROM SELECTKBEST")
    print(f"{'='*60}")

    print(f"Best k values per fold: {fold_best_k_values}")
    if fold_best_k_values:
        print(f"Average best k: {np.mean(fold_best_k_values):.1f} ± {np.std(fold_best_k_values):.1f}")
        print(f"k range: {min(fold_best_k_values)} - {max(fold_best_k_values)}")

    # Create consensus from fold selections
    if fold_selected_features:
        all_features = set()
        for features in fold_selected_features:
            all_features.update(features)
        
        # Count how many times each feature was selected
        feature_counts = {}
        for feature_idx in all_features:
            count = sum(1 for features in fold_selected_features if feature_idx in features)
            feature_counts[feature_idx] = count

        # Select features that appear in at least 3/5 folds
        consensus_threshold = 3
        consensus_features = [idx for idx, count in feature_counts.items() if count >= consensus_threshold]
        
        # Fallback: if no features meet threshold, use union
        if not consensus_features:
            consensus_features = list(all_features)
            print(f"Warning: No features met {consensus_threshold}/5 threshold. Using union of all selections.")
        
        consensus_features = sorted(consensus_features)
        print(f"Consensus features: {len(consensus_features)} selected")
        print(f"Features selected in ≥{consensus_threshold} folds: {len([c for c in feature_counts.values() if c >= consensus_threshold])}")
        
        # Show feature stability analysis
        print(f"\nFeature Stability Analysis:")
        stability_counts = {}
        for count in feature_counts.values():
            stability_counts[count] = stability_counts.get(count, 0) + 1
        for stability, count in sorted(stability_counts.items(), reverse=True):
            print(f"  {count} features selected in {stability}/5 folds")
    else:
        print("No features selected in any fold!")
        consensus_features = []

    # ===============================================================
    # 4) Final evaluation on test set with consensus features
    # ===============================================================
    print(f"\n{'='*60}")
    print("FINAL EVALUATION ON TEST SET")
    print(f"{'='*60}")

    if consensus_features:
        # Scale full training data and test data
        scaler_final = StandardScaler()
        X_train_final_scaled = scaler_final.fit_transform(X_temp)
        X_test_final_scaled = scaler_final.transform(X_test_raw)

        # Apply consensus feature selection
        X_train_final = X_train_final_scaled[:, consensus_features]
        X_test_final = X_test_final_scaled[:, consensus_features]

        print(f"Using {len(consensus_features)} consensus features for final evaluation")

        # Final results storage
        final_results = {}

        for clf in classifiers:
            print(f"\nFinal evaluation: {clf.__class__.__name__}")
            
            # Tune and train on full training set with consensus features
            tuned_clf_final = get_tuned_model(clf, X_train_final, y_temp)
            tuned_clf_final.fit(X_train_final, y_temp)

            # Test set predictions
            if hasattr(tuned_clf_final, 'predict_proba'):
                y_test_proba = tuned_clf_final.predict_proba(X_test_final)[:, 1]
            else:
                y_test_proba = tuned_clf_final.decision_function(X_test_final)

            y_test_pred = tuned_clf_final.predict(X_test_final)

            # Calculate test metrics
            test_auc = roc_auc_score(y_test, y_test_proba)
            test_acc = accuracy_score(y_test, y_test_pred)

            # Bootstrap CI for test AUC
            test_mean_auc, test_lower, test_upper, test_valid = bootstrap_final_model_auc_ci(
                y_test, y_test_proba, n_bootstrap=1000
            )

            final_results[clf.__class__.__name__] = {
                'test_auc': test_auc,
                'test_acc': test_acc,
                'auc_ci_lower': test_lower,
                'auc_ci_upper': test_upper
            }

            print(f"  Test AUC: {test_auc:.4f} [95% CI: {test_lower:.4f}, {test_upper:.4f}]")
            print(f"  Test Accuracy: {test_acc:.4f}")

    # ===============================================================
    # 5) COMPREHENSIVE RESULTS SUMMARY
    # ===============================================================
    print(f"\n{'='*80}")
    print("COMPREHENSIVE SELECTKBEST FEATURE SELECTION RESULTS")
    print(f"{'='*80}")

    print(f"\nDATASET SUMMARY:")
    print(f"- Total samples: {len(X_raw)} (Train: {len(X_temp)}, Test: {len(X_test_raw)})")
    print(f"- Original features: {X_raw.shape[1]}")
    print(f"- Consensus features: {len(consensus_features) if consensus_features else 0}")
    print(f"- Class distribution: LRR={np.sum(y)} ({100*np.sum(y)/len(y):.1f}%), Non-LRR={len(y)-np.sum(y)} ({100*(len(y)-np.sum(y))/len(y):.1f}%)")

    # SelectKBest parameter summary
    if fold_best_k_values:
        print(f"\nSELECTKBEST PARAMETER SUMMARY:")
        print(f"- k values tested per fold: varies (auto-determined)")
        print(f"- Best k per fold: {fold_best_k_values}")
        print(f"- Average optimal k: {np.mean(fold_best_k_values):.1f} ± {np.std(fold_best_k_values):.1f}")
        print(f"- k stability: {len(set(fold_best_k_values))} unique values across 5 folds")

    # Cross-validation performance summary
    print(f"\nCROSS-VALIDATION PERFORMANCE (Mean ± Std):")
    print("-" * 80)
    print(f"{'Classifier':<20} {'Train AUC':<15} {'Val AUC':<15} {'Train Acc':<15} {'Val Acc':<15} {'Avg Features':<12}")
    print("-" * 80)

    for clf_name, results in cv_results.items():
        if results['train_aucs']:  # Only if we have results
            train_auc_mean = np.mean(results['train_aucs'])
            train_auc_std = np.std(results['train_aucs'])
            val_auc_mean = np.mean(results['val_aucs'])
            val_auc_std = np.std(results['val_aucs'])
            train_acc_mean = np.mean(results['train_accs'])
            train_acc_std = np.std(results['train_accs'])
            val_acc_mean = np.mean(results['val_accs'])
            val_acc_std = np.std(results['val_accs'])
            avg_features = np.mean(results['n_features'])
            
            print(f"{clf_name:<20} {train_auc_mean:.3f}±{train_auc_std:.3f}     {val_auc_mean:.3f}±{val_auc_std:.3f}     {train_acc_mean:.3f}±{train_acc_std:.3f}     {val_acc_mean:.3f}±{val_acc_std:.3f}     {avg_features:.1f}")

    # Final test performance
    if consensus_features and final_results:
        print(f"\nFINAL TEST PERFORMANCE WITH 95% BOOTSTRAP CONFIDENCE INTERVALS:")
        print("-" * 70)
        print(f"{'Classifier':<20} {'Test AUC [95% CI]':<25} {'Test Accuracy':<15}")
        print("-" * 70)
        
        for clf_name, results in final_results.items():
            test_auc = results['test_auc']
            test_lower = results['auc_ci_lower']
            test_upper = results['auc_ci_upper']
            test_acc = results['test_acc']
            
            print(f"{clf_name:<20} {test_auc:.3f} [{test_lower:.3f},{test_upper:.3f}]     {test_acc:.3f}")

        # Best performing classifier
        best_test_clf = max(final_results.keys(), key=lambda x: final_results[x]['test_auc'])
        best_test_auc = final_results[best_test_clf]['test_auc']
        best_test_acc = final_results[best_test_clf]['test_acc']
        best_test_lower = final_results[best_test_clf]['auc_ci_lower']
        best_test_upper = final_results[best_test_clf]['auc_ci_upper']
        
        print(f"\nBEST TEST PERFORMANCE: {best_test_clf}")
        print(f"Test AUC: {best_test_auc:.4f} [95% CI: {best_test_lower:.4f}, {best_test_upper:.4f}]")
        print(f"Test Accuracy: {best_test_acc:.4f}")

        # Feature selection effectiveness
        print(f"\nFEATURE SELECTION EFFECTIVENESS:")
        print(f"- Dimensionality reduction: {X_raw.shape[1]} → {len(consensus_features)} features ({100*len(consensus_features)/X_raw.shape[1]:.1f}% retained)")
        print(f"- Feature stability: {len([c for c in feature_counts.values() if c >= 3])}/{len(feature_counts)} features appeared in ≥3/5 folds")
    else:
        print("\n⚠️ No consensus features found or final evaluation skipped.")

    print(f"\n{'='*80}")
    print("SELECTKBEST FEATURE SELECTION PIPELINE COMPLETE")
    print(f"{'='*80}")
    
    # Optional: Return results for further analysis
    results_dict = {
        'cv_results': cv_results,
        'final_results': final_results if 'final_results' in locals() else None,
        'consensus_features': consensus_features,
        'fold_best_k_values': fold_best_k_values,
        'feature_counts': feature_counts if 'feature_counts' in locals() else None
    }

Initial clinical data: 128 patients
Final matched data: 128 patients
DATASET INFORMATION:
Total samples: 128
Total features: 103
Class distribution: LRR=47, Non-LRR=81
LRR percentage: 36.7%

DATA SPLIT:
Train/val samples: 102
Test samples: 26
Train/val class dist: LRR=37, Non-LRR=65
Test class dist: LRR=10, Non-LRR=16

SELECTKBEST FEATURE SELECTION WITH 5-FOLD CROSS-VALIDATION

FOLD 1/5
----------------------------------------
Running SelectKBest feature selection...
Testing k values: [5, 10, 15, 20, 25, 30, 40, 50]

Testing k=5...
  CV Score: 0.6350 (±0.0503)

Testing k=10...
  CV Score: 0.6411 (±0.0376)

Testing k=15...
  CV Score: 0.6730 (±0.0274)

Testing k=20...
  CV Score: 0.5781 (±0.0684)

Testing k=25...
  CV Score: 0.5818 (±0.0852)

Testing k=30...
  CV Score: 0.5710 (±0.0436)

Testing k=40...
  CV Score: 0.5604 (±0.0727)

Testing k=50...
  CV Score: 0.5450 (±0.0601)

Best k: 15 with CV score: 0.6730
Selected 15 features: [  2   5   6   7   8   9  11  15  52  61  68  73  85  8

---